In [1]:
%pylab inline
# import seaborn as sns
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy import sparse
from scipy.sparse.linalg import svds

Populating the interactive namespace from numpy and matplotlib


In [3]:
items_df = pd.read_csv('item_metadata.csv',usecols=[0]) #useless don't run cause real data is missing them

# Final code - Ibrahim

In [2]:
train_df = pd.read_csv('train.csv',sep=',',usecols=['user_id','action_type','reference'])#,'impressions'])#,engine='python')

In [3]:
train_df.dropna(inplace=True)

In [4]:
user_ids=train_df.user_id.unique().tolist()

In [5]:
hotel_ids_tmp=train_df.reference.unique().tolist()
hotel_ids=list()
for it_id in hotel_ids_tmp:
    if(it_id.isdigit()):
        hotel_ids.append(it_id)
len(hotel_ids)

363874

In [25]:
shape

(730803, 927142)

In [39]:
utility_mtrx=np.matrix(np.zeros(shape),np.uint8) #left this to show how even a uint8 can still cause memory error

MemoryError: 

### Training starts here

In [13]:
utility_csr_mtrx=csr_matrix(shape,dtype=np.float32)

In [16]:
#np rewrite
for _, row in train_df.iterrows():
    u_id=row['user_id']
    it_id=row['reference']
    if(not it_id.isdigit()):
        continue
    val=0
    action=row['action_type']
#     print(it_id)
#     print(type(it_id))
#     print(it_id in utility_df.columns)
#     if it_id in hotel_ids:
#         print(row)
    if action == "clickout item":
        val=5
    elif action == "interaction item rating":
        val=2
    elif action == "interaction item info":
        val=3
    elif action == "interaction item image":
        val=2
    elif action == "interaction item deals":
        val=3
    u_idx=user_ids.index(u_id)
    try:
        it_idx=hotel_ids.index(it_id)
    except ValueError:
        print(it_id)
        continue
    
    utility_csr_mtrx[u_idx,it_idx]=utility_csr_mtrx[u_idx,it_idx]+val

G:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\scipy\sparse\compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [17]:
utility_csr_mtrx[0,0]

20.0

In [ ]:
# https://stackoverflow.com/questions/8955448/save-load-scipy-sparse-csr-matrix-in-portable-data-format

In [18]:
np.savez("csr_utility_mtrx", data=utility_csr_mtrx.data, indices=utility_csr_mtrx.indices,indptr=utility_csr_mtrx.indptr, shape=utility_csr_mtrx.shape)

## Loading starts from here

In [6]:
loader = np.load("csr_utility_mtrxScipy.npz") #loading
utility_csr_mtrx= csr_matrix((loader['data'], loader['indices'], loader['indptr']),shape=loader['shape'])

In [15]:
utility_csr_mtrx[0,0]

20.0

In [7]:
utility_csr_mtrx.shape

(730803, 363874)

In [ ]:
# https://antoinevastel.com/machine%20learning/python/2016/02/14/svd-recommender-system.html

## For Computing SVD model

In [8]:
from scipy.sparse.linalg import svds
def SVD(matrix, num_dim=6): #https://github.com/ZWMiller/svdRec/blob/master/svdRec/svdRec.py
        """
        Computes the matrix decomposition for the data, setting U, V, and s to the output matrices.
        This is based on 'svds', a singular value decomposition library designed to work with
        scipy sparse matrices.
        Input: num_dim (int): the number of dimensions for truncating SVD. 
        Returns: None
        """
        U,s,V = svds(matrix,k=num_dim) # I leave k default to 6 cause memory
        return U,s,V

In [9]:
def user_item_predict(U,V,userIdx,itemIdx):
        """
        Helper function to return user-item similarity via dot product.
        Input: U,V, Idx of user, item (int), Idx of item
        Returns: Dot product of user-item matrix elements (float)
        """
        return np.dot(U[userIdx],V.T[itemIdx])

In [10]:
def recommends_for_user(U,V, userIdx, num_recom=25, show_similarity=False):
        """
        Loops through all items and finds the overlap in the reduced vector space
        with the input user. Returns a list of the ID's for the most similar items, 
        as determined by the angular overlap. 
        Inputs: 
        user Idx (int): The user to return recommended items for, 
        num_recom (int): The number of items to recommend,
        show_similarity (bool): If True, the returned values are tuples (itemID, dot product)
        
        Returns:
        ItemIdx (int)
        Dot Product (float) [if show_similarity = True]
        """
        recs = []
        for item in range(V.T.shape[0]):
            recs.append((item,user_item_predict(U,V,userIdx,item)))
        if show_similarity:
            final_rec = [(i[0],i[1]) for i in sorted(recs,key=lambda x: x[1],reverse=True)]
        else:
            final_rec = [i[0] for i in sorted(recs,key=lambda x: x[1],reverse=True)]
        if num_recom==None:
            return final_rec
        
        return final_rec[:num_recom]

####  testing if it works

In [28]:
k=int(min(utility_csr_mtrx.shape)*0.4)
k

145549

In [30]:
u,s,v=SVD(utility_csr_mtrx)

In [50]:
recommends_for_user(u,v, 0)

[43134,
 5256,
 2419,
 7044,
 12027,
 12370,
 5255,
 13751,
 16791,
 4797,
 16307,
 7509,
 7510,
 54582,
 26725,
 153499,
 7518,
 523,
 5941,
 13834,
 63805,
 10553,
 24224,
 4796,
 9676]

## Testing starts here

In [16]:
test_df = pd.read_csv('test.csv')
test_df.head(10)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,004A07DM0IDW,1d688ec168932,1541555614,1,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN
1,004A07DM0IDW,1d688ec168932,1541555614,2,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN
2,004A07DM0IDW,1d688ec168932,1541555696,3,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
3,004A07DM0IDW,1d688ec168932,1541555707,4,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
4,004A07DM0IDW,1d688ec168932,1541555717,5,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
5,004A07DM0IDW,1d688ec168932,1541555792,6,clickout item,3241426,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
6,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
7,009RGHI3G9A3,f05ab0de907e2,1541570935,1,interaction item info,7065316,IN,"Nathdwara, India",mobile,NaN,NaN,NaN
8,009RGHI3G9A3,f05ab0de907e2,1541570940,2,clickout item,NaN,IN,"Nathdwara, India",mobile,NaN,10884872|7065316,64|28
9,00Y1Z24X8084,26b6d294d66e7,1541651766,1,clickout item,3843244,PH,"Iloilo City, Philippines",mobile,NaN,2714480|4476010|3843244|3833012|9017890|198100...,74|14|22|38|55|44|28|34|23|27|12|108|19|21|36|...


In [19]:
type(utility_csr_mtrx)

scipy.sparse.csr.csr_matrix

In [11]:
test_df = pd.read_csv('test.csv',usecols=['user_id','session_id','timestamp','step','user_id','action_type','reference','impressions'])

#### Extending user index

In [12]:
testIdx_user_start=len(user_ids)

In [63]:
user_ids[len(user_ids)-1]

'ZYNMLE3MV3LK'

In [13]:
len(test_df.user_id.unique().tolist())

250852

In [14]:
user_ids.extend(test_df.user_id.unique().tolist())

In [16]:
testIdx_user_start

730803

In [15]:
len(user_ids)

981655

In [60]:
type(user_ids)

list

### Extending hotel Index

In [17]:
test_df_noNan=test_df.dropna(subset=['action_type','reference'])

In [18]:
testIdx_hotel_start=len(hotel_ids)

In [19]:
hotel_ids_tmp=test_df_noNan.reference.unique().tolist()
for it_id in hotel_ids_tmp:
    if(it_id.isdigit()):
        if(not(it_id in hotel_ids)):
            hotel_ids.append(it_id)
len(hotel_ids)

394238

In [20]:
testIdx_hotel_start

363874

In [76]:
utility_csr_mtrx.shape

(730803, 363874)

In [78]:
# newShape=(utility_csr_mtrx.shape[0]+len(test_df.user_id.unique().tolist()),utility_csr_mtrx.shape[1])
# newShape

(981655, 363874)

In [21]:
utility_csr_mtrx.resize((len(user_ids),len(hotel_ids)))

In [ ]:
for _, row in test_df_noNan.iterrows():
    u_id=row['user_id']
    it_id=row['reference']
    if(not it_id.isdigit()):
        continue
    val=0
    action=row['action_type']
    if action == "clickout item":
        val=5
    elif action == "interaction item rating":
        val=2
    elif action == "interaction item info":
        val=3
    elif action == "interaction item image":
        val=2
    elif action == "interaction item deals":
        val=3
    u_idx=user_ids.index(u_id)
    try:
        it_idx=hotel_ids.index(it_id)
    except ValueError:
        print(it_id) #hotel doesnt exist
        continue
    
    utility_csr_mtrx[u_idx,it_idx]=utility_csr_mtrx[u_idx,it_idx]+val

G:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\scipy\sparse\compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [ ]:
# save matrix with testing set loaded
np.savez("csr_utility_mtrxTest.npz", data=utility_csr_mtrx.data, indices=utility_csr_mtrx.indices,indptr=utility_csr_mtrx.indptr, shape=utility_csr_mtrx.shape)

## Getting recommendations for test

In [ ]:
# If not already loaded
loader = np.load("csr_utility_mtrxTest.npz") #loading
utility_csr_mtrx= csr_matrix((loader['data'], loader['indices'], loader['indptr']),shape=loader['shape'])

In [ ]:
test_refNan=test_df[test_df['reference'].isnull()].dropna(subset=['impressions'])

In [ ]:
#Won't pass hotel_ids index to this function but will use it globally to save processor jump time
def translate_rec_n_filter(recommandations,impressions):
    res="" #string not array to upload and compare
    hotels=impressions.split("|")
    count=0
    for it_idx in recommandations:
        hotel=hotel_ids[it_idx]
        if hotel in hotels: #exists in impressions
            res=res+" "+hotel
            count=count+1
            if count>=25:
                return res

In [ ]:
u,s,v=SVD(utility_csr_mtrx)

In [ ]:
with open('Gamma_IbrahimAbouelseoud_SVD.csv', 'a') as file:
    totalSize=str(test_refNan.shape[0])
    file.write('user_id'+ ',' + 'session_id'+ ',' +'timestamp'+ ',' +'step'+ ',' +'item_recommendations'+ '\n')
    for i,entry in test_refNan.iterrows():
        rec=translate_rec_n_filter(recommends_for_user(u,v, user_ids.index(entry['user_id']), num_recom=None),entry['impressions'])
        file.write(str(entry['user_id'])+ ',' +str(entry['session_id'])+ ',' +str(entry['timestamp'])+ ',' +str(entry['step'])+ ',' +str(rec))
        if(i%1000==0):
            file.flush()
            print(str(i)+" out of"+totalSize)